###### Load libraries and directories

In [1]:
from IPython.display import display, HTML

In [2]:
# from IPython import get_ipython
from tqdm.notebook import tqdm
import pickle
import os
import pprint
pp = pprint.PrettyPrinter(indent=1)

# Custom modules for debugging
from SliceViewer import ImageSliceViewer3D, ImageSliceViewer3D_1view,ImageSliceViewer3D_2views
from investigate import *

#pd.set_option("display.max_rows", 10)
      
import json
from run_sma_experiment import find_l3_images,output_images
import pprint
from L3_finder import *

# Custom functions
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):        
    with open(filename, 'rb') as input:
        return pickle.load(input)

Using TensorFlow backend.


In [3]:
get_ipython().run_line_magic('tb', '')

No traceback available to show.


In [4]:
cwd = os.getcwd()
print(cwd)
data = '/tf/data'
pickles = '/tf/pickles'
models = '/tf/models'

/tf/smipipeline


In [5]:
# Import modules and config file
configfile = os.path.join(cwd,'config/debug_ES/run_prediction_CV_poorl3.json')
with open(configfile, "r") as f:
        config = json.load(f)
pp.pprint(config)

{'l3_finder': {'cache_dir': '/tf/_cache/',
               'cache_intermediate_results': True,
               'dicom_dir': '/tf/data',
               'model_path_dir': '/tf/models/l3/cv_final',
               'new_tim_dicom_dir_structure': True,
               'output_directory': '/tf/output/cv_poorl3/l3',
               'overwrite': True,
               'save_plots': True,
               'show_plots': False},
 'muscle_segmentor': {'model_path_dir': '/tf/models/muscle/cv_final',
                      'output_directory': '/tf/output/cv_poorl3/ms'}}


## Section 1 - Process final images

## Section 2 - Load each study into subject object
<br>
Subject object defined in L3finder.ingest

In [6]:
config['l3_finder']['new_tim_dicom_dir_structure']

True

In [7]:
# Debug
print("Finding subjects")

subjects = list(
    find_subjects(
        config['l3_finder']["dicom_dir"],
        new_tim_dir_structure=config['l3_finder']['new_tim_dicom_dir_structure']
    )
)

print('Subjects found: ', len(subjects))

Finding subjects
Subjects found:  2367


## Section-3 - check if there are subjects with multiple folders (studies)

In [8]:
%%time
# Find series images
print("Finding series")
series = list(flatten(tqdm((s.find_series() for s in subjects),total=len(subjects))))

Finding series



CPU times: user 290 ms, sys: 149 ms, total: 438 ms
Wall time: 682 ms


In [9]:
print("Total number of series found: ", len(series))

Total number of series found:  3758


In [10]:
%%time
sagittal_series, axial_series, excluded_series = separate_series(series)

Filtering series
CPU times: user 858 ms, sys: 836 ms, total: 1.69 s
Wall time: 2.69 s


In [11]:
print("Length of valid pats: ", len(subjects))
print("Length of sagittal series", len(sagittal_series))
print("Length of axial series", len(axial_series))
print("Length of excluded series", len(excluded_series))
print("Length of all series in dataset", len(series))

Length of valid pats:  2367
Length of sagittal series 1391
Length of axial series 2367
Length of excluded series 0
Length of all series in dataset 3758


### Make sure each subject has at the max only 1 axial and 1 sagittal series

In [ ]:
ax_ids = [ax.subject.id_ for ax in axial_series]
sag_ids = [sag.subject.id_ for sag in sagittal_series]

def find_duplicates(id_list):
    uniques = []
    duplicates = []
    for ids in id_list:
        if ids in uniques:
            duplicates.append(ids)
        else:
            uniques.append(ids)
            
    return uniques,duplicates


ax_u,ax_d = find_duplicates(ax_ids)
sag_u,sag_d = find_duplicates(sag_ids)

print('Ax duplicates: ', ax_d)
print('Sag duplicates: ', sag_d)

In [ ]:
# Find the series objects to investigate
ax_d_series = [ax for ax in axial_series if ax.subject.id_ in ax_d]
sag_d_series = [ax for ax in sagittal_series if ax.subject.id_ in ax_d]

In [ ]:
df_dl= load_object(os.path.join(pickles,'df_final.pkl'))
display(df_dl[df_dl['ID']==ax_d[0]])

## Reconstruct Missing Sagittals

In [ ]:
# By default code filters 0.5mm slices, but I am letting them pass by setting it to 0
constructed_sagittals = construct_series_for_subjects_without_sagittals(
        subjects, sagittal_series, axial_series,thickness_filter=0) 

In [ ]:
print(
        "Series separated\n",
        len(sagittal_series), "sagittal series.",
        len(axial_series), "axial series.",
        len(excluded_series), "excluded series.",
        len(constructed_sagittals), "constructed series.",
    )

In [ ]:
sagittal_series.extend(constructed_sagittals)

In [ ]:
save_object(axial_series,os.path.join(pickles,'axial_curated.pkl'))
save_object(sagittal_series,os.path.join(pickles,'sagittal_curated.pkl'))

## Create MIPS

In [ ]:
print("Creating sagittal MIPS")
mips = create_sagittal_mips_from_series(
        many_series=sagittal_series,
        cache_dir=config['l3_finder'].get("cache_dir", None),
        cache=config['l3_finder'].get("cache_intermediate_results", False),
    )

In [ ]:
save_object(mips,os.path.join(pickles,'mips.pkl'))

## Find L3

In [ ]:
mips = load_object(os.path.join(pickles,'mips.pkl'))

In [ ]:
print("Preprocessing Images")
preprocessed_images = preprocess_images(mips)

# Sagittal mip is redundant, get rid just use preprocessed images
sagittal_mips = [SagittalMIP(i) for i in preprocessed_images]

print("Separate heights for better batching")
mips_by_dimension = group_mips_by_dimension(sagittal_mips)
print("Dimensions in set:", mips_by_dimension.keys())

In [ ]:
save_object(mips_by_dimension,os.path.join(pickles,'mips_by_dimension.pkl'))

## Find L3 - step 2

In [11]:
mips_by_dimension = load_object(os.path.join(pickles,'mips_by_dimension.pkl'))
axial_series = load_object(os.path.join(pickles,'axial_curated.pkl'))
sagittal_series = load_object(os.path.join(pickles,'sagittal_curated.pkl'))

In [12]:
# Get all models in model path dir
models_dir = config['l3_finder']['model_path_dir']

# Get all models in models dir
models_list = sorted([f for f in os.listdir(models_dir) if f.endswith('.h5')])
print(models_list)
folds = len(models_list)

for fold in range(folds):
    model_path = os.path.join(models_dir,models_list[fold])
    print(model_path)

['UNet1D_cv_1_of_5.h5', 'UNet1D_cv_2_of_5.h5', 'UNet1D_cv_3_of_5.h5', 'UNet1D_cv_4_of_5.h5', 'UNet1D_cv_5_of_5.h5']
/tf/models/l3/cv_final/UNet1D_cv_1_of_5.h5
/tf/models/l3/cv_final/UNet1D_cv_2_of_5.h5
/tf/models/l3/cv_final/UNet1D_cv_3_of_5.h5
/tf/models/l3/cv_final/UNet1D_cv_4_of_5.h5
/tf/models/l3/cv_final/UNet1D_cv_5_of_5.h5


In [13]:
runname = 'CV_poorl3'
if __name__ == "__main__":
    for fold in range(folds):
        model_path = os.path.join(models_dir,models_list[fold])
        print("Making predictions for fold ", fold, 'Path: ', model_path)
        prediction_results = []
        prediction_errors = []
        for dimension, sagittal_mips in mips_by_dimension.items():
            dim_group_results,errors = make_predictions_for_sagittal_mips(
                sagittal_mips,
                model_path=model_path,
                shape=dimension
            )
            prediction_results.extend(dim_group_results)
            prediction_errors.extend(errors)

        # Save prediction results
        pred_results_file = 'prediction_results_' + str(fold) + '_' +  runname + '.pkl'
        pred_errors_file = 'prediction_errors_' + str(fold) + '_' +  runname + '.pkl'
        save_object(prediction_results,os.path.join(pickles,pred_results_file))
        save_object(prediction_errors,os.path.join(pickles,pred_errors_file))

Making predictions for fold  0 Path:  /tf/models/l3/cv_final/UNet1D_cv_1_of_5.h5


error drawing line on preprocessed_image for: Z1996094
shape: (125, 512, 1), prediction: 126

error drawing line on preprocessed_image for: Z701477
shape: (185, 512, 1), prediction: 186

error drawing line on preprocessed_image for: Z495526
shape: (215, 512, 1), prediction: 216

error drawing line on preprocessed_image for: Z362374
shape: (888, 512, 1), prediction: 888



Making predictions for fold  1 Path:  /tf/models/l3/cv_final/UNet1D_cv_2_of_5.h5


error drawing line on preprocessed_image for: Z892856
shape: (430, 512, 1), prediction: 431

error drawing line on preprocessed_image for: Z1996094
shape: (125, 512, 1), prediction: 127

error drawing line on preprocessed_image for: Z1140510
shape: (245, 512, 1), prediction: 246

error drawing line on preprocessed_image for: Z495526
shape: (215, 512, 1), prediction: 216

error drawing line on preprocessed_image for: Z878416
shape: (462, 512, 1), prediction: 463

error drawing line on preprocessed_image for: Z1196674
shape: (410, 512, 1), prediction: 411

error drawing line on preprocessed_image for: Z1273303
shape: (495, 512, 1), prediction: 496

error drawing line on preprocessed_image for: Z391986
shape: (420, 512, 1), prediction: 421

error drawing line on preprocessed_image for: Z526993
shape: (282, 512, 1), prediction: 283

error drawing line on preprocessed_image for: Z1239639
shape: (360, 512, 1), prediction: 361

error drawing line on preprocessed_image for: Z913179
shape: (220

Making predictions for fold  2 Path:  /tf/models/l3/cv_final/UNet1D_cv_3_of_5.h5
Making predictions for fold  3 Path:  /tf/models/l3/cv_final/UNet1D_cv_4_of_5.h5


error drawing line on preprocessed_image for: Z495526
shape: (215, 512, 1), prediction: 216



Making predictions for fold  4 Path:  /tf/models/l3/cv_final/UNet1D_cv_5_of_5.h5


error drawing line on preprocessed_image for: Z627604
shape: (455, 512, 1), prediction: 458

error drawing line on preprocessed_image for: Z1996094
shape: (125, 512, 1), prediction: 127

error drawing line on preprocessed_image for: Z495526
shape: (215, 512, 1), prediction: 217



### Save l3 prediction results

In [14]:
axial_series = load_object(os.path.join(pickles,'axial_curated.pkl'))

In [15]:
len(axial_series)

2367

In [16]:
# Load prediction_results pickle files
runname = 'CV_poorl3'

output_dir = config["l3_finder"]["output_directory"]
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

prediction_results_list = sorted([f for f in os.listdir(pickles) if (f.startswith('prediction_results_') and f.endswith(runname+'.pkl'))])
folds = len(prediction_results_list)
print(folds)

if __name__ == "__main__":
    for fold in range(folds):
        pred_file = os.path.join(pickles,prediction_results_list[fold])
        prediction_results = load_object(pred_file)
        print('Total predictions: ',len(prediction_results))
        print('Building L3 images for fold: ', fold)
        l3_images = build_l3_images(axial_series, prediction_results)
        print('Total images: ',len(l3_images))
        # Don't run this unless you have new L3 results
        print("Outputting L3 images for fold: ", fold)
        # Clears pixel data from memory aafter outputting
        output_dir = os.path.join(config["l3_finder"]["output_directory"],str(fold))
        
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        l3_images = output_images(
        l3_images,
        args=dict(
            output_directory=output_dir,
            should_plot=config["l3_finder"]["show_plots"],
            should_overwrite=config["l3_finder"]["overwrite"],
            should_save_plots=config["l3_finder"]["save_plots"]
        ))

5
Total predictions:  2363
Building L3 images for fold:  0
Total images:  2363
Outputting L3 images for fold:  0
Slow unless axial images already loaded...


 39%|███▉      | 923/2363 [02:32<19:08,  1.25it/s]  /tf/smipipeline/l3finder/output.py:142: RuntimeWarning: overflow encountered in ushort_scalars
  assert img_max + (-img_min) < 65536
100%|██████████| 2363/2363 [13:16<00:00,  2.97it/s]


Total predictions:  2347
Building L3 images for fold:  1
Total images:  2347
Outputting L3 images for fold:  1
Slow unless axial images already loaded...


100%|██████████| 2347/2347 [13:10<00:00,  2.97it/s] 


Total predictions:  2367
Building L3 images for fold:  2
Total images:  2367
Outputting L3 images for fold:  2
Slow unless axial images already loaded...


 58%|█████▊    | 1366/2367 [03:42<03:47,  4.39it/s] /tf/smipipeline/l3finder/output.py:142: RuntimeWarning: overflow encountered in ushort_scalars
  assert img_max + (-img_min) < 65536
100%|██████████| 2367/2367 [13:25<00:00,  2.94it/s]


Total predictions:  2366
Building L3 images for fold:  3
Total images:  2366
Outputting L3 images for fold:  3
Slow unless axial images already loaded...


 59%|█████▉    | 1401/2366 [03:47<07:05,  2.27it/s] /tf/smipipeline/l3finder/output.py:142: RuntimeWarning: overflow encountered in ushort_scalars
  assert img_max + (-img_min) < 65536
 96%|█████████▌| 2274/2366 [09:17<00:46,  1.97it/s]/tf/smipipeline/l3finder/output.py:142: RuntimeWarning: overflow encountered in ushort_scalars
  assert img_max + (-img_min) < 65536
100%|██████████| 2366/2366 [13:26<00:00,  2.93it/s]


Total predictions:  2364
Building L3 images for fold:  4
Total images:  2364
Outputting L3 images for fold:  4
Slow unless axial images already loaded...


100%|██████████| 2364/2364 [13:12<00:00,  2.98it/s] 


## Find mean L3 prediction

In [14]:
import csv
import sys
from collections import defaultdict
from pathlib import Path

from L3_finder import L3Image
from l3finder import ingest

from compare_best_to_manual_l3_and_seg import MinimalPrediction, MinimalResult

In [15]:
def load_l3_predictions(l3_prediction_dir,nfolds):
    subject_id_col = 0
    pred_in_px_col = 1
    predictions = defaultdict(list)

    for fold_index in range(0, nfolds):
        csv_dir = os.path.join(l3_prediction_dir,str(fold_index))
        csv_path = Path(csv_dir,'l3_prediction_results.csv')
        with open(csv_path) as csvfile:
            reader = csv.reader(csvfile)
            next(reader)

            for row in reader:
                subid = row[subject_id_col].split('-')[0]
                predictions[subid].append(float(row[pred_in_px_col]))

    return predictions

def calc_mean_predictions(all_predictions: defaultdict):
    result = {}
    for subject_id, prediction_list in all_predictions.items():
        result[subject_id] = np.mean(prediction_list)
    return result

def find_subjects_w_preds(predictions, all_subjects):
    subject_ids_w_preds = set(predictions.keys())
    return [s for s in all_subjects if s.id_ in subject_ids_w_preds]

def load_l3_images_from_predictions(mean_predictions, subjects_w_preds,axials,sagittals):
    l3_images = []

    for subject in subjects_w_preds:
        sagittal_series = [s for s in sagittals if s.subject.id_ == subject.id_][0]
        axial_series = [a for a in axials if a.subject.id_ == subject.id_][0]
        l3_images.append(
            L3Image(
                axial_series=axial_series,
                sagittal_series=sagittal_series,
                prediction_result=MinimalResult(
                    MinimalPrediction(
                        predicted_y_in_px=mean_predictions[subject.id_]
                    )
                )
            )
        )
    return l3_images

In [16]:
axial_series = load_object(os.path.join(pickles,'axial_curated.pkl'))
sagittal_series = load_object(os.path.join(pickles,'sagittal_curated.pkl'))

In [17]:
runname = 'CV_poorl3'
prediction_results_list = sorted([f for f in os.listdir(pickles) if (f.startswith('prediction_results_') and f.endswith(runname+'.pkl'))])
folds = len(prediction_results_list)
print(folds)

5


In [18]:
if __name__ == "__main__":
    all_predictions = load_l3_predictions(config["l3_finder"]["output_directory"],folds)
    mean_predictions = calc_mean_predictions(all_predictions)  

In [19]:
if __name__ == "__main__":
    subjects_w_preds = find_subjects_w_preds(mean_predictions, list(ingest.find_subjects(config['l3_finder']["dicom_dir"])))
    l3_images = load_l3_images_from_predictions(mean_predictions, subjects_w_preds, axial_series, sagittal_series)

In [20]:
save_object(l3_images,os.path.join(pickles,'l3_images_cv.pkl'))
save_object(mean_predictions,os.path.join(pickles,'mean_predictions.pkl'))

### Handle Outlier Cases

In [6]:
l3_images = load_object(os.path.join(pickles,'l3_images_cv.pkl'))

In [21]:
infile  = 'poorl3.csv'
df_poorl3 = pd.read_csv(infile, index_col=False)

print('Total number of outliers for manual L3 detection: ', len(df_poorl3))
l3_absent = df_poorl3.loc[df_poorl3['L3slice'].isnull(),'ID'].values.tolist()
print('Cases with L3 not present: ', len(l3_absent))
l3_present = df_poorl3.loc[~df_poorl3['L3slice'].isnull(),'ID'].values.tolist()
print('Cases with manually identified L3s: ', len(l3_present))

#sagittal_mips_valid = [sagittal_mip for sagittal_mip in sagittal_mips if sagittal_mip.subject_id not in df_poorl3.ID.values]

Total number of outliers for manual L3 detection:  45
Cases with L3 not present:  15
Cases with manually identified L3s:  30


In [22]:
display(df_poorl3.head(10))

,ID,L3slice,comment
0,Z1996094,NaN,this appears to be just additional images thro...
1,Z1262030,NaN,this is a lung MIP and chest only
2,Z705523,NaN,this is a lung MIP and chest only
3,Z1762020,99.0,this is a lung MIP - is there not a soft tissu...
4,Z1195730,NaN,this is a lung MIP and chest only
5,Z1286046,NaN,this is a lung MIP and chest only
6,Z418856,101.0,this is a lung MIP - is there not a soft tissu...
7,Z1450371,NaN,this is a lung MIP and chest only
8,Z687529,NaN,this is a lung MIP and chest only
9,Z1436778,NaN,this is a lung MIP and chest only


In [23]:
# Get rid of outliers without proper L3 images
print('Total l3_images: ', len(l3_images))
l3_images = [l3_image for l3_image in l3_images if l3_image.subject_id not in l3_absent]
print('Total l3_images after outlier removal: ', len(l3_images))

Total l3_images:  2367
Total l3_images after outlier removal:  2352


In [24]:
l3_images_out = [l3_image for l3_image in l3_images if l3_image.subject_id in l3_present]
print(len(l3_images_out))

30


In [25]:
# Create Manual Predictions DICT
manual_predictions = {}
for i in range(len(l3_images_out)):
    subject_id = l3_images_out[i].subject_id
    start = l3_images_out[i].prediction_index[1].first_axial_pos
    end = l3_images_out[i].prediction_index[1].last_axial_pos
    counts = l3_images_out[i].prediction_index[1].axial_image_count
    z_pos = l3_images_out[i].prediction_index[1].predicted_z_position
    idx = l3_images_out[i].prediction_index[1].l3_axial_image_index

    new_idx = df_poorl3.loc[df_poorl3['ID']==subject_id,'L3slice'].values[0]
#     print('subject_id: ', subject_id, 'new_idx: = ',new_idx)    
#     break

    thick = abs((start-end)/counts)
    
    new_z_pos = 0
    if start > end:
        new_z_pos = start - thick*new_idx
    else:
        new_z_pos = start + thick*new_idx
    
    manual_predictions[subject_id] = new_z_pos
    
#     if new_z_pos < 0:
#         print('thick: ', thick)
#         print('start: ', start)
#         print('end: ', end)
#         print('old idx: ', idx)
#         print('old z: ', z_pos)
      
#         print('new idx: ', new_idx)
#         print('new z: ', new_z_pos)
    
#     print('\n')
    
print(manual_predictions)

{'Z1762020': 1857.4905660377358, 'Z717870': 923.4038461538462, 'Z362374': -316.2702702702703, 'Z890238': -1221.0151515151515, 'Z1722108': -1435.9842767295597, 'Z1256768': 1287.5316455696202, 'Z1119985': -236.65, 'Z418856': -1391.9749774932861, 'Z678707': 1446.3648648648648, 'Z670302': 1336.5046728971963, 'Z1302641': -172.61810952044547, 'Z489517': 1716.9052287581699, 'Z320930': 1914.5, 'Z1263347': 1710.5, 'Z1000800': 1521.5645161290322, 'Z1274627': -127.5995145631068, 'Z5745': 1810.8378305970245, 'Z1029886': 1574.323943661972, 'Z832424': 1411.3809523809523, 'Z441830': 1695.107142857143, 'Z1041413': 1744.8521126760563, 'Z567376': 1971.142857142857, 'Z1332420': 1643.7625899280574, 'Z1221549': 1329.0418648905804, 'Z511402': -423.79398148148147, 'Z357478': 1580.855828220859, 'Z1211258': -310.29166666666663, 'Z486791': -455.30769230769226, 'Z837620': 1739.2115331607183, 'Z627309': -215.90540540540542}


In [26]:
mean_predictions_old = mean_predictions.copy()
for key,value in manual_predictions.items():
    print(' Old prediction: ', mean_predictions[key],' new prediction: ', manual_predictions[key])
    mean_predictions[key] = round(manual_predictions[key],1)

 Old prediction:  821.8  new prediction:  1857.4905660377358
 Old prediction:  207.6  new prediction:  923.4038461538462
 Old prediction:  528.0  new prediction:  -316.2702702702703
 Old prediction:  495.6  new prediction:  -1221.0151515151515
 Old prediction:  690.2  new prediction:  -1435.9842767295597
 Old prediction:  168.2  new prediction:  1287.5316455696202
 Old prediction:  226.6  new prediction:  -236.65
 Old prediction:  736.8  new prediction:  -1391.9749774932861
 Old prediction:  107.2  new prediction:  1446.3648648648648
 Old prediction:  554.0  new prediction:  1336.5046728971963
 Old prediction:  348.75  new prediction:  -172.61810952044547
 Old prediction:  459.0  new prediction:  1716.9052287581699
 Old prediction:  155.6  new prediction:  1914.5
 Old prediction:  95.8  new prediction:  1710.5
 Old prediction:  322.6  new prediction:  1521.5645161290322
 Old prediction:  377.4  new prediction:  -127.5995145631068
 Old prediction:  427.4  new prediction:  1810.837830597

In [30]:
l3_images = load_l3_images_from_predictions(mean_predictions, subjects_w_preds, axial_series, sagittal_series)

In [31]:
# Get rid of outliers without proper L3 images
print('Total l3_images: ', len(l3_images))
l3_images = [l3_image for l3_image in l3_images if l3_image.subject_id not in l3_absent]
print('Total l3_images after outlier removal: ', len(l3_images))
save_object(l3_images,os.path.join(pickles,'l3_images_cv_outliershandled.pkl'))

Total l3_images:  2367
Total l3_images after outlier removal:  2352


In [ ]:
# Verify manual L3 selection was successfuly by comparing slice location in image to manual selection csv file

In [32]:
l3_images_manual = [l3_image for l3_image in l3_images if l3_image.subject_id in l3_present]

In [37]:
print(' Subject ID ',' Slice Z position ',' Slice Index','  Manual Index')
for i in range(len(l3_images_out)):
    subject_id = l3_images_out[i].subject_id
    z_pos = l3_images_out[i].prediction_index[1].predicted_z_position
    idx = l3_images_out[i].prediction_index[1].l3_axial_image_index
    print(subject_id,'     ', np.round(z_pos,0),'      ', idx, '     ',df_poorl3.loc[df_poorl3['ID']==subject_id,'L3slice'].values[0])

 Subject ID   Slice Z position   Slice Index   Manual Index
Z1762020       1429.0        158       99.0
Z717870       763.0        42       30.0
Z362374       -682.0        146       55.0
Z890238       -1367.0        112       79.0
Z1722108       -1729.0        158       100.0
Z1256768       1317.0        34       40.0
Z1119985       -376.0        79       42.0
Z418856       -1727.0        159       101.0
Z678707       1514.0        21       36.0
Z670302       1048.0        106       67.0
Z1302641       -375.0        348       147.0
Z489517       1624.0        115       92.0
Z320930       1914.0        0       33.0
Z1263347       1710.0        0       37.0
Z1000800       1276.0        65       36.0
Z1274627       -376.0        74       38.0
Z5745       1440.0        1300       491.0
Z1029886       1614.0        25       33.0
Z832424       1453.0        31       40.0
Z441830       1471.0        55       30.0
Z1041413       1462.0        212       75.0
Z567376       1703.0        130    

### Segment L3 Axial Images and Calculate Muscle Area

In [ ]:
l3_images = load_object(,os.path.join(pickles,'l3_images_cv_outliershandled.pkl'))

In [29]:
from compare_best_to_manual_l3_and_seg import seg_model_configs
from compare_best_to_manual_l3_and_seg import do_segmentation_cv

In [88]:
config["muscle_segmentor"]['model_path_dir']

'/tf/models/muscle/cv_final'

In [89]:
if __name__ == "__main__":
    configs = seg_model_configs(config["muscle_segmentor"]['model_path_dir'])
    smas,average_masks,tableless_images = do_segmentation_cv(configs, l3_images)
    print("Done")

- Loading l3 axial images


2352it [01:29, 26.31it/s]
  1%|▏         | 35/2352 [00:00<00:06, 343.83it/s]

- Removing table
 - Taking care of images that have a rescale value of -1024, as opposed to -2048 for cannon images


  4%|▍         | 101/2352 [00:00<00:02, 1008.45it/s]

  - zeroing images


...

 - Taking care of maximum value: default sigma above which to denoise is 1


100%|██████████| 2352/2352 [00:58<00:00, 40.29it/s]

  - removing table



2352it [00:19, 119.62it/s]


- Thresholding images
- Normalizing images
- Resizing images
 Segmenting using all model configs
segmenting for model {'model_path': '/tf/models/muscle/cv_final/combined_2020-02-18_dice_fold_0.h5'}
segmenting for model {'model_path': '/tf/models/muscle/cv_final/combined_2020-02-18_dice_fold_1.h5'}
segmenting for model {'model_path': '/tf/models/muscle/cv_final/combined_2020-02-18_dice_fold_2.h5'}
segmenting for model {'model_path': '/tf/models/muscle/cv_final/combined_2020-02-18_dice_fold_3.h5'}
segmenting for model {'model_path': '/tf/models/muscle/cv_final/combined_2020-02-18_dice_fold_4.h5'}
Finding Average Masks
Finding sma
Done


In [90]:
from imageio import imsave

def output_sma_results(output_dir, l3_images, tableless_images, average_masks, smas):
    os.makedirs(output_dir, exist_ok=True)

    csv_filename = os.path.join(output_dir, "areas-mm2_by_subject_id.csv")
    with open(csv_filename, "w") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["subject_id", "area_mm2", "sagittal_series", "axial_series"])
        print('Saving Segmentation Results in ', output_dir)
        index = 0    
        for mask, sma, l3_image, tableless_image in zip(average_masks, smas,l3_images, tableless_images):
            index += 1
            base = os.path.join(output_dir, str(index) + "_" + l3_image.subject_id)
            imsave(base + "_CT.tif", tableless_image.astype(np.float32))
            imsave(base + "_muscle.tif", mask * np.iinfo(np.uint8).max)

            row = [
                l3_image.subject_id,
                sma.area_mm2,
                l3_image.sagittal_series.series_name,
                l3_image.axial_series.series_name,
            ]
            csv_writer.writerow(row)
        print('Total exams outputted: ', index)

In [91]:
output_sma_results(config["muscle_segmentor"]['output_directory'], l3_images, tableless_images, average_masks, smas)

Saving Segmentation Results in  /tf/output/cv_poorl3/ms
Total exams outputted:  2352
